In [1]:
import os
import random
import yaml
import pickle
import sys
sys.path.append('./tsfm_public')
# Third Party
from transformers import (
    EarlyStoppingCallback,
    PatchTSMixerConfig,
    PatchTSMixerForPrediction,
    Trainer,
    TrainingArguments,
)
import numpy as np
import pandas as pd
import torch
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfm_public.toolkit.util import select_by_index
from read_data import loadData
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

c:\Users\02213R744\anaconda3\envs\patch_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_train, testDatasets, expectedRulDatasets = loadData()

In [3]:
drop_col = ['setting_1', 'setting_2', 'setting_3'] 
data_clean = [data.drop(columns = drop_col) for data in data_train]

In [4]:
df_all = pd.DataFrame()
for i in range(4):
    df_all =pd.concat([df_all, data_clean[i]])

In [5]:
for engine in list(set(df_all['engine'])):
    max_cycle = df_all[df_all['engine'] == engine]['cycle'].max() 
    condition = (df_all['engine'] == engine) & (df_all['cycle'] > max_cycle - 25)
    df_all.loc[condition, 'label'] = 1

In [6]:
df_all = df_all.fillna(0)

In [7]:
df_all['label'].value_counts()

label
0.0    152974
1.0      7385
Name: count, dtype: int64

In [8]:
df_engine_1 = df_all.drop(columns=['index'], errors='ignore')

In [9]:
correlation_matrix = df_engine_1.drop(columns=['engine']).corr()
high_corr_columns = correlation_matrix[abs(correlation_matrix['label'] )> 0.03].index.tolist()
df_sample = df_engine_1[high_corr_columns]
df_sample = df_sample.drop(columns=['label'])

In [10]:
df_sample.shape

(160359, 9)

In [11]:
target_col = None 

In [12]:
with open("./config.yaml", "r") as file:
    configs = yaml.safe_load(file)

num_workers = configs['num_workers'] # Reduce this if you have low number of CPU cores
batch_size = configs['batch_size']  # Reduce if not enough GPU memory available
context_length = configs['context_length'] 
forecast_horizon = configs['forecast_horizon']  # 8 hours 
patch_length = configs['patch_length'] 
# target_col = configs['target_columns']

timestamp_column = "cycle"

In [13]:
df_sample.loc[:, 'index'] = df_sample.index
data = df_sample.copy()

In [14]:
timestamp_column = 'index'

In [15]:
id_columns = []
forecast_columns = [col for col in data.columns if col not in timestamp_column]
train_start_index = None  # None indicates beginning of dataset
train_end_index = int(len(data)*configs['train_data_split']) 
# we shift the start of the validation/test period back by context length so that the first validation/test timestamp is immediately following the training data
valid_start_index = int(len(data)*configs['train_data_split']) - context_length 
valid_end_index = int(len(data)*configs['train_data_split']) + int(len(data)*configs['valid_data_split']) 
test_start_index = int(len(data)*configs['train_data_split']) + int(len(data)*configs['valid_data_split']) - context_length 
test_end_index = len(data) 

train_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=train_start_index,
    end_index=train_end_index,
)
valid_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=valid_start_index,
    end_index=valid_end_index,
)
test_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=test_start_index,
    end_index=test_end_index,
)
tsp = TimeSeriesPreprocessor(
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    scaling=True,
)

tsp.train(train_data)

train_dataset = ForecastDFDataset(
    tsp.preprocess(train_data),
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
valid_dataset = ForecastDFDataset(
    tsp.preprocess(valid_data),
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
test_dataset = ForecastDFDataset(
    tsp.preprocess(test_data),
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)

In [16]:
with open("../model/patch_model/timeseriesprocessor.pkl", "wb") as file:
    pickle.dump(tsp, file)

In [17]:
config = PatchTSMixerConfig(
        context_length=context_length,
        prediction_length=forecast_horizon,
        patch_length=patch_length,
        num_input_channels=len(forecast_columns),
        patch_stride=patch_length,
        d_model=48,
        num_layers=3,
        expansion_factor=3,
        dropout=0.5,
        head_dropout=0.7,
        mode="common_channel", # change it `mix_channel` if we need to explicitly model channel correlations
        scaling="std",
    )
model = PatchTSMixerForPrediction(config=config)
train_args = TrainingArguments(
    output_dir="./output/",
    overwrite_output_dir=True,
    learning_rate=configs['learning_rate'],
    num_train_epochs=configs['epochs'],
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=num_workers,
    report_to="tensorboard",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir="./logs/",  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"],
)
# Create a new early stopping callback with faster convergence properties
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=5,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.001,  # Minimum improvement required to consider as improvement
)

In [18]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
)
print("\n\nDoing training")
trainer.train()
trainer.evaluate(test_dataset)
save_dir = configs['patch_foundation_model_path']
os.makedirs(save_dir, exist_ok=True)
trainer.save_model(save_dir)
print(f'Saved the foundation model at {configs["patch_foundation_model_path"]}')

c:\Users\02213R744\anaconda3\envs\patch_env\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(




Doing training


100%|██████████| 1203/1203 [02:14<00:00,  1.32s/it]

{'loss': 0.4878, 'learning_rate': 0.0, 'epoch': 1.0}


                                                   
100%|██████████| 1203/1203 [02:42<00:00,  7.40it/s]


{'eval_loss': 0.7541259527206421, 'eval_runtime': 28.2042, 'eval_samples_per_second': 568.283, 'eval_steps_per_second': 5.708, 'epoch': 1.0}
{'train_runtime': 162.5109, 'train_samples_per_second': 739.876, 'train_steps_per_second': 7.403, 'train_loss': 0.48778759075618244, 'epoch': 1.0}


100%|██████████| 241/241 [00:14<00:00, 16.30it/s]

Saved the foundation model at ../model/patch_model
